In [1]:

# load embedding

from gensim.models.keyedvectors import KeyedVectors
word2vec= KeyedVectors.load_word2vec_format('./embedding/GoogleNews-vectors-negative300.bin', binary=True)
print("word2vec load successfully")
emoji2vec =KeyedVectors.load_word2vec_format('./embedding/emoji2vec.bin', binary=True)
print("emoji2vec load successfully")
emoticon2vec = KeyedVectors.load_word2vec_format('./embedding/emoticon2vec.txt', binary=False)
print("emoticon2vec load successfully")    
print('Loading complete')

word2vec load successfully
emoji2vec load successfully
emoticon2vec load successfully
Loading complete


In [2]:
#load dictionary
import pickle
emoji_dict =pickle.load(open('./data/SST2/emoji_dict.p','rb'))
print("emoji dictionary load successfully")

emoji dictionary load successfully


In [3]:
import pandas as pd
import numpy as np

In [4]:
train = pd.read_csv('./data/SST2/train.csv')
dev = pd.read_csv('./data/SST2/dev.csv')
test = pd.read_csv('./data/SST2/test.csv')

In [40]:
for i in np.arange(0.1,1,0.1):
    X_train_sub,temp1= train_test_split(train,test_size=1-i,stratify=train['label'])
    print(X_train_sub.shape)
    path = '/Users/nihaozheng/Desktop/NLP/project/model/pytorch-sentiment-classification-master/data/subset/'
    pd.DataFrame(X_train_sub).to_csv(path+str(i)+'train.csv')
    

(4595, 2)
(9191, 2)
(13787, 2)
(18382, 2)
(22978, 2)
(27574, 2)
(32169, 2)
(36765, 2)
(41361, 2)


In [5]:
train.shape,dev.shape,test.shape

((45957, 2), (2419, 2), (12095, 2))

In [6]:
train.head()

,text,label
0,While I'm breathing - I love and believe. emoj...,0
1,emoji428 ufe0f emoji1143 ufe0f emoji202 prod...,0
2,u2014 faye is that u emoji1606,0
3,https://t.co/shgiye2olp | | follow us emoji11...,1
4,gives me so much joy to teach these beautiful ...,1


In [7]:
emoji_dict

{'\\U0001f30c': 'emoji0',
 '\\U0001f549': 'emoji1',
 '\\U0001f30a': 'emoji2',
 '\\U0001f30f': 'emoji3',
 '\\U0001f30e': 'emoji4',
 '\\U0001f30b': 'emoji5',
 '\\U0001f396': 'emoji6',
 '\\U0001f926\\u200d\\u2640\\ufe0f': 'emoji7',
 '\\U0001f465': 'emoji8',
 '\\U0001f9d8\\U0001f3fc\\u200d\\u2642\\ufe0f': 'emoji9',
 '\\U0001f462': 'emoji10',
 '\\U0001f463': 'emoji11',
 '\\U0001f468\\u200d\\U0001f469\\u200d\\U0001f466\\u200d\\U0001f466': 'emoji12',
 '\\U0001f499': 'emoji13',
 '\\U0001f498': 'emoji14',
 '\\U0001f392': 'emoji15',
 '\\U0001f493': 'emoji16',
 '\\U0001f492': 'emoji17',
 '\\U0001f491': 'emoji18',
 '\\U0001f490': 'emoji19',
 '\\U0001f497': 'emoji20',
 '\\U0001f496': 'emoji21',
 '\\U0001f495': 'emoji22',
 '\\U0001f494': 'emoji23',
 '\\U0001f945': 'emoji24',
 '\\U0001f944': 'emoji25',
 '\\U0001f947': 'emoji26',
 '\\U0001f941': 'emoji27',
 '\\U0001f940': 'emoji28',
 '\\U0001f943': 'emoji29',
 '\\U0001f942': 'emoji30',
 '\\U0001f949': 'emoji31',
 '\\U0001f937\\U0001f3fe\\u200d\\u2642\

In [8]:
#build reverse dict, i.e emoji*** : \U****
reverse_emoji_dict = {}
for key,item in emoji_dict.items():
    reverse_emoji_dict[item]=key

In [9]:
reverse_emoji_dict['emoji0']

'\\U0001f30c'

In [10]:
train['label'].unique()

array([0, 1, 3, 5, 4, 2])

# 1.1 transform text into embedding (sum)

In [11]:
import time

In [12]:
X_train,y_train = train['text'],train['label']
X_test,y_test = test['text'],test['label']
X_dev,y_dev =dev['text'],dev['label']

In [13]:
# set all unseen word to 0
def individual2vec(word):
    #print(word)
    if word in emoticon2vec.vocab:
        print(word)
        return emoticon2vec[word]
    elif word in reverse_emoji_dict.keys():
        d = reverse_emoji_dict[word]
        d = d.encode('utf-8')
        d = d.decode('unicode-escape')
        if d in emoji2vec.vocab:
            print(d)
            return emoji2vec[d]
        else:
            return np.zeros(300)
            
    elif word in word2vec.vocab:
        return word2vec[word]
    else:
        return np.zeros(300)

    

In [14]:
def sentence2vec(s):
    #sum over the embedding
    s = s.split()
    return sum(list(map(individual2vec,s)))

In [15]:
def data2vec(data):
    return np.array(list((map(sentence2vec,data))))

In [16]:
start = time.time()

X_train = data2vec(X_train)
X_test = data2vec(X_test)
X_dev=data2vec(X_dev)

end = time.time()
print("the transformation takes {0:.2f} minutes".format((end-start)/60 ))

😍
⛪
🙏🏻
😭
👉
🇨🇩
🎶
😒
😂
😂
😂
😘
👏🏽
🎉
👄
👉
😘
🖤
🌈
:)
:)
😩
🤦
😝
😥
😥
😟
😟
💪
👻
😂
🤗
🔞
😊
🔆
👍
🦃
👌
🍂
📸
💋
💋
💋
💋
💋
💋
💋
💕
☕
✨
💫
✨
💫
😍
😇
😍
🦄
🌈
🏳
🌈
🎂
🍰
🎈
🎉
👍🏽
👍🏽
👍🏽
👍
👑
🌟
😂
😂
😂
🍫
😋
🐝
💛
💙
🎧
🙏
🎸
📍
😋
🔞
🌹
🌹
🌹
💐
💐
🍏
👑
🍏
🌹
🌹
🌹
🌹
🌹
🌹
🌹
🌹
🌹
🌹
🌹
👆
👆
👆
👆
👆
👆
👆
💐
💐
🍏
👑
🍏
🌹
🌹
🌹
🌹
🌹
🌹
🌹
🌹
🌹
🌹
🌹
🌹
🌹
👇
👇
👇
👇
👇
👇
👇
🍸
📷
💕
🦄
🦄
😘
📼
🍁
😍
😻
😽
😹
😽
😻
😏
😍
😍
😍
☘
☘
☘
☘
☘
☘
👉
👹
👹
🙋🏻
🌹
🌹
🌹
☕
🎶
🐦
🍃
🍁
🌿
🍂
🍃
🍁
🌿
🍂
🍃
🌹
🌹
🇩🇿
😍
😍
😍
🗣
💨
🎌
🙆
💀
😂
😘
😍
😜
🔥
😜
😍
🚺
🌐
🌟
🌟
😋
😋
😍
😍
💜
🎃
🐤
🍓
👋
🤜
🤛
👉
👈
😂
😂
😂
💀
💀
💀
🌷
👠
💋
😍
😍
😍
😘
😘
😘
😍
👅
👅
💦
💦
💦
🖤
😋
👍
😍
👍
😊
👏
🍋
✅
🍋
✅
🎂
😍
😘
😄
😂
🍓
👉
🔥
⚡
⚡
💟
💞
💋
💋
💋
💋
☂
✨
😆
👏
✨
⚾
👀
💓
😂
😂
😂
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😡
😍
😍
😋
😋
😋
😋
😋
🔞
💪
😊
💯
☮
☮
☮
☮
☮
☮
☮
☮
☮
☮
❗
❗
💚
🐈
😺
🌍
🌎
🌏
🗺
💖
💖
🖱
🖱
💔
🃏
😉
🔥
💗
:)
🌹
🌹
🌹
☕
🎶
🐦
🍃
🍁
🌿
🍂
🍃
🍁
🌿
🍂
🍃
🌹
🌹
🎓
💎
📸
🤡
🤣
💜
💜
🏆
;)
🌷
🌹
🥀
🌺
🌸
🌼
🌻
🔃
🔃
🔃
🔃
👇
✨
💥
😍
🙏🏼
🙏🏼
🔊
💾
👇
👇
👇
💕
🦊
☕
😍
🤣
😍
😍
😍
👊
💙
🦋
🦋
🌷
💘
🐇
🐞
🦋
😎
😎
😎
🥂
🥂
🥂
😭
😭
😭
😭
😭
😭
😭
😭
😭
😭
😭
😭
😭
🎉
🎉
🎉
🎉
🎊
🎊
🎊
🎊
😇
😇
💦
💦
📍
📍
😉
😁
😊
🙌
🙏🏻
🤓
😘
😇
📷
🙏🏽
👌
👏
👀
👅
😈
✊
💦
🔥
💋
💕
😘
🤗


👆
👆
👆
👆
👆
👆
⏩
⏪
👋
👉
👉
💗
💗
🙏
🐶
👱
😩
😋
👹
👹
✨
✨
🎄
😇
😲
🙏🏽
🍓
😍
😍
😍
😊
😍
💞
:)
🌈
💓
😉
💞
🙏
😈
😈
💕
💕
💕
💭
💭
💕
😍
😘
💪🏼
🤔
😜
➖
😊
:)
😃
😃
🔞
💖
💡
😘
🦄
🦄
✨
✨
🌟
💙
🔥
👌🏼
🙈
💛
💛
💛
💛
💛
💛
🤣
😂
🙌
💀
😋
😋
🍆
💘
🌋
👓
💎
🇫🇷
👉
👉
:)
🦉
🇹🇷
🇵🇹
😂
😍
😘
😉
👍
😙
😙
😙
😉
😘
🔥
🔥
🔥
🔥
💯
💯
💯
🙏🏻
➖
➖
😜
😃
😃
😜
💦
💦
🌹
🌹
🌹
🍓
😎
🍨
😭
🤣
🤣
🤣
🌟
🌟
🌟
⏩
⏪
🔑
🌹
🌹
😻
🔥
💖
👧
😇
💐
💪🏼
💪🏼
💪🏼
💦
💦
😊
😊
😊
🕯
🥀
☄
💡
💡
💫
💪🏻
✨
🖤
😉
📿
🕉
😚
😄
💖
💖
💖
💪
🌞
🌞
🌞
✨
💚
😍
😎
🔥
🔥
💖
💕
🤗
😇
🏄🏻
🏄🏻
🏄🏻
🏄🏻
🏄🏻
🏄🏻
🍓
💙
🔞
😀
😍
💗
😁
💕
💕
😍
📞
🙏
💙
💙
💙
✝
💙
😃
🔴
😁
👍
🍑
👌
🔥
💙
👇🏼
👇🏼
👇🏼
🔥
🔥
🍑
🍆
🍓
👣
😜
😉
💕
🙃
😭
🦋
✨
✨
✨
😭
😭
😭
💕
💕
💕
😭
😭
💓
💀
🔥
🔥
🔥
🔥
🔥
🔥
😂
👏
👏
💛
😁
😁
😃
💋
⭐
🌞
🌞
🌞
🌞
👊
😎
👊
💖
😎
🌴
🌸
😊
🤣
😍
👉
😊
😣
💜
👑
😍
🤔
🤔
💓
💓
🎶
💦
💦
💦
🔥
🔥
🔥
🌷
🐝
🐣
💐
🐇
🐞
🦋
🌹
💻
📕
💖
😍
🤓
🔪
😅
💋
💋
🙏🏽
🤢
👍
🥘
🍷
✨
💫
😁
💕
💗
🎨
🎨
💗
💚
💪
🙏
🔗
🌵
🙌
💍
=)
🙏
🎼
🍓
😝
💋
⭐
🔥
🔥
🤗
😍
💞
🐚
🐠
💖
💙
💛
💜
👫
🇯🇵
⛩
🌸
🍒
😀
🙏
🙏
💙
🙏
😍
💙
🏖
🏡
🌴
🏃
💖
💖
💖
😉
🍕
🍴
😍
💦
💦
💦
🔥
🔥
🔥
🍺
🎓
😍
🙏
😂
🇯🇲
😂
💋
💋
💋
💋
💋
💋
💋
⛪
🙏🏻
😙
😔
➖
🔞
😮
😉
😅
😂
🤣
👋
😂
😂
😂
⭐
⭐
🌹
🔥
🔥
🤣
😍
😜
🔥
😜
😍
🚺
🌐
😄
😄
🎉
🎉
👭
👶
😝
👿
❕
🔥
🌍
👉
👈
📸
📸
💖
🍑
💞
🔥
🔥
🔥
😱
😊
❣
❣
❣
😂
😂
😂
😍
😍
👇
👇
😍
😍
🖤
🙏🏻
🍜
🤔
🆕
🌺
💕
🌺
💕
🌺
💕
🌺
💕
😘
😍
😍
😍
😍
😘
💕
💞
😂
🏘
💓
😍
🤗
💎
🙌
💋
🙌
🔥
💙
✅
🔥
💙
🍒
🍒

🔥
😜
😍
🆕
👀
⭐
🔥
🔥
🐻
💖
💞
💞
💞
💖
🙌
💓
🙌
🌞
🌍
🌹
🌸
🌻
🌟
✨
💫
💛
💔
☮
☯
🙏
💞
🔗
💪
😍
😉
😍
😍
😍
:)
🌚
🌟
😡
🍭
🍼
💦
🍆
🍑
💍
💅🏼
💵
🌈
😬
😷
😂
😂
👍🏾
👻
🎉
🙌
😎
🍺
👉
😀
😁
😂
😍
👶
💛
⭐
⭐
⭐
⭐
👀
🐶
😍
😊
😁
🖼
🎨
👽
🔥
💓
💓
💓
💓
🍓
🔊
💾
👇
👇
👇
🙏🏿
😂
😂
😍
😍
😍
🙏🏽
😍
😍
💡
🎥
🎬
📸
📺
🔥
🌊
💡
🎥
🎬
📸
📺
:)
💋
💋
💋
💋
💋
💋
💋
😍
👮
👮
🚓
😂
👐
🍑
👐
🆘
🐾
😁
🤣
😂
🙈
👉
👈
🤑
💦
💦
💦
🔥
🔥
🔥
💚
😍
💖
💖
💖
😍
😍
😍
😍
🔥
🔥
🎄
🎟
🗄
🍛
🍳
😋
📱
💦
🌷
🌷
🌷
🌷
🌷
😂
👀
😂
😂
😂
😂
👀
👀
👀
🔱
💇
💯
💪
💪🏾
🌌
🙏🏾
🙏🏾
☘
☘
☘
☘
☘
☘
👉
😘
😍
💕
🔊
🎶
😎
😍
💪🏽
🥀
💯
🙏🏾
😈
😈
😈
😈
😘
😘
😘
😘
😘
😘
😎
🔎
🗝
🔒
👍
😁
😁
😁
😁
😩
😀
😻
👍
👌
🌹
🙏🏼
🌸
😊
:)
💫
💫
💪
😊
🌿
✅
🌀
💚
💛
✨
🙌
😁
😜
😭
😭
😂
🌴
🍦
💙
💚
💛
💜
📷
💦
💦
💦
🔥
🔥
🔥
😂
😂
😂
😂
🤗
🙌
💪
🌞
🤗
🤗
💀
💦
😘
💕
:)
🌅
💖
🔞
🐎
💪
🙏
🌹
👉
💋
💋
😛
😍
🌈
🗣
🤗
🌸
🍃
🌸
🍃
😊
😊
😊
🌞
😂
🚇
😍
😘
😍
😎
😩
😭
😍
🍆
💦
💥
🍓
🌀
🌞
🌀
🙄
😬
💎
👍
😈
😈
🙃
💖
🌍
🤘
🤘
🤘
😍
📠
🐘
🤤
🤤
💚
😍
👌
💥
🔥
👉
😇
😇
🎶
😏
🌴
✨
✨
🌴
💪
💪
🗳
😂
😍
🤝
🍓
💥
🙈
💚
💚
💖
💞
😳
🍑
🍆
😍
💋
💦
😋
👨
👦
🤕
🏎
💨
🚛
🚍
😢
😢
😢
🙏
😍
😍
😍
😋
🍑
😋
😋
😍
💖
🦋
🦋
📲
🖤
💕
🖤
💕
🖤
💕
🖤
💕
🌸
💕
🌸
💕
🌸
💕
🌸
💕
🌸
💕
😍
💜
🐼
😍
😍
😍
💚
💪
🙏
🔗
🍁
🤖
👽
🤖
👽
🌹
🌹
🌹
💐
💐
🍏
👑
🍏
🌹
🌹
🌹
🌹
🌹
🌹
🌹
🌹
🌹
🌹
🌹
👆
👆
👆
👆
👆
👆
👆
💐
💐
🍏
👑
🍏
🌹
🌹
🌹
🌹
🌹
🌹
🌹
🌹
🌹
🌹
🌹
🌹
🌹
👇
👇
👇
👇
👇
👇
👇
🔁
🔁
💯
😍
😍
😍
😍
👉
👉
🤘
🤙
💪
😎
👀
👍
😞
😞
🤷
🤷
😀
😃


👚
😎
💚
💛
🇺🇸
🔥
🔥
👍
👍
✅
✅
✅
✅
✅
✅
👉
👈
💎
😍
👇
😥
😥
😞
😞
😂
😩
👎🏼
💫
⛑
⛑
💫
💫
✨
✨
💫
✨
✨
💫
✨
💫
✨
💫
✨
💫
✨
💫
💥
💥
😋
👨
👦
😤
🤙
🌞
📸
💎
👇
👇
👇
🌟
🌟
💢
💢
💢
🎬
⏩
⏩
⏩
⏪
⏪
⏪
👉
👉
👈
👈
❣
😘
😆
➖
➖
😂
😂
😂
🙏🏾
💦
💦
💘
💘
💞
💞
💞
💞
💞
💞
😮
💯
☠
☠
💃🏽
👀
😜
🦋
🍒
🐣
🐰
😄
🍫
🔃
🔃
🔃
🔃
😘
💞
😭
☹
💯
👀
👍
😀
💀
💀
💀
💀
💀
💀
💀
💀
💀
💀
😘
😘
😊
😄
😚
👨
👩
👶
🌵
🌾
🌿
🍀
🌞
🤗
🙃
🙃
🙃
🙃
🌷
💗
😂
😂
😂
🐣
🐥
⭐
😍
😍
🌊
🏖
🍃
📷
✨
💀
💀
😆
😀
💐
🌴
💗
💎
😍
💝
🍸
💰
💞
💍
😎
🙏
🐰
💟
💐
🐍
🐯
🐒
😂
😂
😂
🌞
😙
🍃
💕
🌟
💜
🌟
🔥
🆕
🔥
🆕
😘
😍
😍
😘
⏩
⏩
⏩
⏪
⏪
⏪
🔞
🌟
🌟
⏩
⏩
⏩
⏪
⏪
⏪
😭
😂
✋
😊
🌸
😅
😅
😅
😅
😍
😍
😍
😍
😍
🔥
😭
😍
💕
🎥
⚽
😥
🐺
🔥
😢
😘
😍
😇
😎
🙄
😂
💙
🤠
🎻
🎸
🎸
🎻
🤠
🚫
🙏
🙏
💦
💦
💦
🌈
🌈
🌈
😎
😊
😃
👓
👯
👯
😍
😘
📌
📌
👆
👆
👆
👆
👆
👆
👆
👆
👆
👆
😜
😏
💦
💦
💦
💋
💋
🤦
🎮
🎮
🔞
💦
💦
💦
🔥
🔥
🔥
🍓
👀
💙
💚
💛
💜
💝
💝
💜
💛
💚
💙
😍
🇮🇪
💜
💛
💚
💥
🏡
😂
🙄
🐶
🐾
👶
😁
🌛
😊
📉
📃
🎉
🏅
🍆
🍆
😍
🦉
🇹🇷
🇵🇹
😂
😍
😘
😉
👍
😂
🌹
🐰
🍓
😁
💚
😂
💗
😋
👉
👉
👉
👈
👈
👈
🔥
🔥
🔥
🔥
😍
😍
👣
👣
💖
💖
💖
💖
😋
:)
💖
🌟
😠
🔞
😳
😱
😍
🍑
😏
⏩
⏩
⏩
⏪
⏪
⏪
💎
💕
💕
💕
🌟
🌟
💋
🤦
🙃
😂
😂
🔥
🔥
🔥
🔥
✅
✅
✅
✅
😎
🐶
🍻
🚧
🐍
👅
😍
🌅
💪
👑
📷
🔥
🔥
😛
😊
🔥
🔥
😍
💃🏻
💕
🏃
🙌
✨
💃🏼
😀
👍🏻
💙
💙
💙
💙
🍨
😎
👇
💯
:)
🎉
🎉
🎉
🎉
🎉
🎉
🎉
🔥
😽
🔞
🌅
🌅
😍
😜
🔥
😜
😍
🚺
🌐
🔆
💯
🏈
🏃
🏃
🏃
🏃
🙏
🕶
😎
🌴
😊
😃
🌞
🚪
📈
📈
📈
✅
🔲
💕
😄
🔝
😎
🇨🇦
🇺🇸
🌹
👑
👑
💕
💙
🌷
🌷
💙


📝
🎊
🎊
🎊
🎼
🎼
🎼
🎼
🎊
💖
💖
💃
💃
🆒
💃
💃
🚶
🚶
🚶
🚶
💖
💖
😀
😉
😇
😍
💪
💪
🍲
👍
🙏🏽
😍
😍
😋
🌟
🌟
🌟
😍
😋
😋
😍
🔥
🔥
🎧
🎧
🙃
😘
📸
🌹
🌹
🌹
☕
🎶
🐦
🍃
🍁
🌿
🍂
🍃
🍁
🌿
🍂
🍃
🌹
🌹
😔
😔
👋🏽
😅
🙏🏻
🙏🏻
🙏🏻
🚨
😋
😋
😋
😍
😋
😘
💖
🌹
🌹
🙊
😘
😂
😂
🙈
💜
💜
🌹
:)
😩
💯
🤙
🤙
🍻
😭
🎉
🎊
🤘
🌠
🤗
😁
🐵
😁
💖
💝
🇩🇪
😍
😜
😘
💖
💗
💖
🐼
😆
😂
🤙
:)
😍
😜
🔥
😜
😍
🚺
🌐
😂
😍
😍
😘
😘
😘
😘
😊
🙏🏿
🌹
😀
📷
💋
💋
💋
💋
💋
💋
💋
🙃
🤣
🤣
🤣
⭐
🌟
💫
⚡
🌟
🌹
✨
🌹
✨
🍓
😈
😂
🌹
🌹
🌹
☕
🎶
🐦
🍃
🍁
🌿
🍂
🍃
🍁
🌿
🍂
🍃
🌹
🌹
🍆
🍆
😍
💜
✨
😁
📖
✝
💀
💀
:)
🔥
🔥
🙏🏼
😩
😍
💎
🙃
😍
😜
🔥
😜
😍
🚺
🌐
🌴
🌴
🌴
🌊
🐳
🐬
🐠
🐟
🦀
🐡
🐙
😅
😱
😵
😵
💚
💙
✊🏾
🙌
💯
😍
💋
💋
💋
💋
💋
💋
💋
🎊
🎊
🎊
💚
👑
💚
🎊
🎊
🎊
🎊
🎊
🎊
🎊
🎊
🎊
🎊
🎊
👆
👆
👆
👆
👆
👆
👆
💚
👑
💚
🎊
🎊
🎊
🎊
🎊
🎊
🎊
🎊
🎊
🎊
🎊
🎊
🎊
👇
👇
👇
👇
👇
😊
😊
🌡
🌡
🌡
💚
😭
😈
😈
❓
🔁
🚂
🤣
😂
🐐
💝
🔝
👑
😏
🌸
☕
🎧
😳
😳
😎
😏
🤤
👌
😍
😈
🔗
🙏🏻
🙏🏻
🙏🏻
👉
💛
💚
💙
💜
🏳
🌈
🏳
🌈
💜
🌶
🌶
🌶
💚
👑
💚
🌶
🌶
🌶
🌶
🌶
🌶
👆
👆
👆
👆
👆
👆
👆
💚
👑
💚
🌶
🌶
🌶
🌶
🌶
🌶
🌶
🌶
👇
👇
👇
👇
👇
🙏🏻
😍
😊
🔝
🖤
😈
💦
💻
🚨
🚨
😲
😲
😲
👻
🤡
🤡
🤡
🤡
🤡
🤡
✅
🌹
😈
💦
🖕
🌎
🌊
🌊
👻
😉
🎀
😃
😁
👉
🔞
💿
🎧
😎
🔥
💣
🔥
💣
💿
😎
😉
💿
🎧
⏩
⏩
⏩
💪
🌻
🤹
✨
✨
✨
😍
🎼
😂
😂
😂
😍
🔥
✨
🔹
🔹
🎧
🔥
🏈
👌
💣
😁
☕
👌🏽
💋
📲
😘
😍
😍
😍
😍
😎
😎
📷
👑
👑
😍
:)
😉
🐢
😍
😍
⭐
🤣
🤣
🤣
🤣
🤣
🤣
🔞
:)
👍
😱
😈
📺
👻
👹
🙌
👆
😋
💯
😹
😹
😹
😹
😂
🚨
🚨
🚨
🚨
💜
💕
💥
👀
🔒
😍
😘
😄
🌃
🌃
🇯🇵
🌼
🏯
⛩
💋


...

🙏🏼
🙏🏼
🙏🏼
💪
💪
💪
💯
💯
💪
💪
🙏🏼
🙏🏼
💪
💪
💯
💯
👙
🍷
📚
🌹
💋
🔥
🔥
😎
🌹
🌼
🌻
🌷
🌷
🌷
🍰
🍹
🎊
🎉
😭
🖤
👉
🖤
🖤
🖤
🖤
🖤
🖤
🖤
🖤
🖤
🖤
🖤
🌈
💪
💪
😍
😍
😳
👇
⏩
⏪
🙏
💣
😍
⏩
⚽
😳
💓
👠
🌟
🐶
💙
😩
😇
😂
😂
😂
😋
😁
😘
🔥
🔥
🔥
🔥
🔥
🔥
🔥
🔥
🔥
🔥
🔥
🔥
😂
😂
😂
8)
🙄
👌🏾
👌🏾
👸
🖌
🆒
😍
😍
👇
👇
😍
😍
😍
😍
😍
😘
😘
😘
😘
😘
😘
😘
🌈
😍
💦
💦
💦
🌶
🌶
🌶
💚
👑
💚
🌶
🌶
🌶
🌶
🌶
🌶
👆
👆
👆
👆
👆
👆
👆
💚
👑
💚
🌶
🌶
🌶
🌶
🌶
🌶
🌶
🌶
👇
👇
👇
👇
👇
😈
😫
👌
😊
🙌
👢
🙄
😍
😝
🌎
🔞
😂
😂
✊
📕
🎶
🌎
🌞
💕
😍
💦
👉
🔞
😉
😊
🤗
🙂
😘
💪🏾
💪🏾
🤔
😘
🌃
😍
🍻
📷
🔞
😂
😅
🙏🏽
😭
😩
💕
😈
👅
🙌
🔃
🎦
🆓
🔥
🆓
🆓
🆓
🆓
🎦
🔊
🔊
💀
💥
💥
👇
👇
💜
💛
💗
💙
💉
💊
💸
💳
🤑
🆕
🆕
🆕
❗
➕
💲
❗
📣
📣
💨
💨
💨
💖
💜
💙
🖤
💚
💜
🖤
💚
💝
💜
💖
💚
🖤
💖
🖤
💜
💙
💜
💚
💜
💗
💜
💝
💙
💚
💜
💗
💜
💝
💜
🖤
💝
🖤
💗
💛
💝
💝
💛
💙
💛
💝
💝
💚
💛
💝
💝
💗
💝
💙
💛
💙
💖
💙
💜
💝
🖤
💗
💛
💗
🖤
💚
💖
💙
💝
🖤
🖤
🖤
💙
💗
💛
💚
💜
:')
💕
💞
💗
✨
🔥
🔥
➕
➕
➕
😍
😆
3️⃣
🔑
📝
📈
👥
🎧
😘
😘
🔥
🔥
🔥
💋
🖤
🐢
⛰
👉🏾
👉🏾
🌡
🌶
😱
☠
💀
😍
😅
🎁
😂
😍
😍
😜
💥
🔥
💥
🔥
💥
💦
💦
💦
😚
😘
😍
💋
💦
🔥
💥
💦
💫
💋
💋
💋
🔁
:)
😳
🌎
🔽
🔽
🔽
🔽
🔽
🔽
🔽
🔽
🔽
🔽
😍
😜
🔥
😜
😍
😍
💦
🔫
😈
😈
😈
🎶
✨
😘
😂
😍
💋
😘
🎉
🔝
🎀
💦
😉
🐶
🌿
🍊
🇲🇽
😉
💪
💪
💪
💪
📷
💟
🍓
💋
😘
😍
💘
🌹
😲
😻
👍
🙏
💪
🙌
🙌
🙌
☕
🎁
👮
😭
😭
😭
😂
👌🏻
🤔
💭
💪🏻
✨
💦
💦
💦
🔥
🔥
🔥
🐻
🎺
🎼
🎷
💜
:)
😀
😍
😍
😍
💰
🔞
🔥
🔥
😁
🤑
😒
😍
😍
😊
💘
💁
💁
😔
👍
👍
👍
👍
👍
👍
👍
👍
👍
👍
👍
👍
👍
💃
💃


❌
💐
💙
🎶
🎼
🙏🏻
😢
⭐
🔥
🔥
🔞
😺
🙌
👻
🐾
😊
🙂
💚
💯
🔥
😍
😛
💁🏽
😳
😂
😂
😂
😂
😍
✨
💫
✨
💫
🎬
🎥
🎥
💋
💋
💋
💋
💋
💋
💋
😂
😂
😂
😂
😂
😂
😂
😭
😁
😊
💙
💛
💜
💚
🔞
👼
🦅
🃏
🃏
🃏
😘
😝
👅
😘
🤣
🔥
🔥
🔥
🔥
🔥
🔥
🔥
🔥
🔥
🔥
🔥
🔥
😎
🌧
😍
😍
🔴
🔴
😍
🙌
👏
🔃
💽
💯
😘
🤗
📣
😍
💜
😍
🙂
😂
🔥
🔥
😍
📢
😍
😍
😍
🆕
👍
📲
💀
💀
😍
💕
💞
😊
✨
😏
:)
💔
💜
😍
😘
🎤
🎼
💦
💦
💦
🔥
🔥
🔥
🔔
🔔
🔔
😀
💝
😍
😍
😍
🔥
😈
🔥
😈
🔥
😈
🔥
😝
😍
🙏
😫
😫
😎
😃
😍
🙏
🇺🇸
😀
🎼
👏
😌
🤤
😻
😜
👶
🙏
🕺
🎉
🤗
💋
💞
🤔
😂
🤗
🤗
🤗
🤗
💦
🍑
💦
🍑
😍
😍
😍
😆
📰
💀
🔨
⛓
🏹
⛏
📽
🎞
☠
🌞
🦋
🌻
🦋
🌻
🦋
🌻
🦋
🦋
✨
😊
🌎
✨
😇
😊
🌸
🌼
🌻
🎥
🏀
🔥
👉
😢
😜
✅
👍🏿
💇🏿
💕
👍
💋
💋
💋
💋
💋
💋
💋
🌻
😁
🔁
👍
😈
🙏
🍔
🍴
😍
😂
😂
😂
😂
😂
😂
😂
😂
😂
🏒
😘
😈
✅
👌
🎉
🎉
💓
😃
🍷
⚽
💥
👁
💰
😎
😎
😎
😍
👍
😎
🌱
💙
🇵🇱
💞
🍀
👁
👋
👉
👈
🤳
🚦
💒
🚩
👉
👈
😂
🔥
😄
🔥
🔥
🐙
✨
✨
✨
🕉
💓
🙏🏻
👬
❣
🎥
🔝
😘
😘
😘
🦋
🌻
👻
😥
🐣
😊
😎
💋
🥂
🛍
🌟
🌟
🌟
🌟
🌟
😍
😜
🔥
😜
😍
🌐
😜
😜
🎧
💚
💛
🐒
🏔
🌲
🍃
🎶
🎧
🎵
🙌
💙
😉
:)
⚽
💗
💋
💗
😀
👍
⭐
😍
😍
😍
😍
😍
👌
😍
👌
🎾
🎉
💥
📷
🎉
🎊
🎉
🎊
🙏🏽
😈
👅
💦
😊
📷
👑
😻
💝
🎂
🎉
🎉
🤙
⏩
⏩
⏪
⏪
😍
👏
👀
🍀
💏
😘
👈
👈
👈
👈
👏
💫
😍
😙
💓
💖
💖
💕
💕
🤦
🙄
😂
💪🏽
🌿
🍃
👑
🌟
😍
💕
🌵
🔥
🍰
🙏🏻
🙏🏻
🍭
➰
🔜
🐶
💛
🐰
🌈
💗
🔊
💾
👇
👇
👇
😘
🎉
✨
😍
😜
🔥
😜
😍
🚺
🌐
💋
💋
💋
💋
💋
💋
💋
😍
🔥
😍
🔥
😍
😍
😍
😍
👉
💚
👄
💞
👍
💋
💝
👄
💞
👍
💋
💝
👄
💞
👍
💋
💝
😂
💜
💦
😍
😏
😏
😄
😄
😄
💙
;)
😈
😈
👑
;)
✝
👍
😍
😜
🔥
😜
😍
🚺
🌐
💣
💣

🇺🇸
🇺🇸
🙌
✨
😍
🔥
😍
🔥
😍
😁
🎊
🎊
🎉
👊
💕
😋
🌈
👑
👑
🔥
✅
🖤
🎶
🎤
💋
💋
💋
💋
💋
💋
💋
🌟
🔥
🆕
🔥
🆕
🙌
🍫
🍫
😂
🏆
🔥
💫
😍
😍
😍
😍
😍
😍
😍
😍
😍
😍
😍
😋
🌻
⏩
⏩
⏪
⏪
😻
😍
👶
💛
⭐
⭐
⭐
⭐
💜
😅
:)
😱
😁
😘
😘
🙏🏼
😉
🐰
🙏
🙏
👨
👦
💕
😂
😂
🌹
🌹
🌹
☕
🎶
🐦
🍃
🍁
🌿
🍂
🍃
🍁
🌿
🍂
🍃
🌹
🌹
😊
💖
🎈
🎈
🎈
🎈
😁
🍃
🌹
💋
🌼
🌼
💋
💦
💦
🍃
🌹
🌹
🍃
❣
❣
❣
❣
🔞
😈
😈
🔞
😍
😍
👊🏻
👉
👉
👉
👉
📩
😘
💕
💋
😎
💜
💜
💜
🍓
💪
💃🏼
🙌
😍
😍
👌
💋
💋
💋
💋
💋
💋
💋
🙂
😍
😍
🍓
✨
😍
✨
🌧
☔
😩
🌧
🌧
😒
🙌
🙌
🎈
✅
⚽
🆚
🏁
➕
📌
📌
👆
👆
👆
👆
👆
👆
👆
👆
👆
👆
💀
🃏
🃏
🃏
🙈
😂
🌊
🔥
🔥
💓
😎
✨
🌟
💫
⭐
✨
🌟
💫
⭐
😍
😍
😍
😎
🤓
😂
😍
😋
😉
😜
😍
😎
🔥
🙈
🙊
💩
😆
🔥
💧
🔥
🔥
💧
🔥
📷
🙏🏼
❌
🙏🏻
🐚
💙
💛
🇵🇷
😃
🍩
🇺🇸
🇮🇱
😈
😈
🍑
💦
💦
💦
🍑
🍆
👀
😋
😉
😳
🔥
🍓
🤣
👩
🔬
👨
🍳
💎
😍
👌🏻
😭
😭
😭
😭
😭
😭
😭
🌸
🌼
🌷
💫
💕
🌊
🙏🏼
🙏🏼
😊
😄
🍆
💦
😜
🔊
💾
👇
👇
👇
🍜
🔊
💾
👇
👇
👇
😍
😜
🔥
😜
😍
🌐
😉
👉
👉
👉
👈
👈
👈
😍
😜
🔥
😜
😍
😊
🌲
🍃
🌹
💋
🌼
🌼
💋
💦
💦
🍃
🌹
🌹
🍃
⏩
⏩
⏩
⏪
⏪
⏪
😍
👍
😍
😍
👍
😍
😍
🎥
🔝
😻
👉
🔥
💣
👀
👉
👈
👌
👌
👌
👌
👌
🎬
🎥
💀
🎭
😂
🙄
📸
✨
🔝
💖
🍓
😍
💚
👍
😉
😘
😘
😘
😛
😛
💞
💞
😂
😂
🔥
🔥
🔥
⏩
⏪
😜
😘
🤘
👉🏽
💥
💥
💯
🙏🏾
🔞
📸
✨
🙌
😍
🍖
💑
🍓
😎
😘
😘
🤔
🤔
🤔
🤔
🤔
🤔
🤔
🤔
🤔
🤔
🤔
🤔
🤔
🤔
🤔
🤔
🤔
🤔
🤔
🤔
🤔
🤔
🤔
🤔
🤔
💋
🎉
☮
🎉
👍
💋
💖
👏
👏
👏
👏
👏
👏
😎
😍
😍
⏩
⏪
😍
🔥
🍁
😭
🌟
🌟
🌟
🌟
🌟
🌟
🌟
🌟
⏩
⏩
➕
➕
😉
👅
😉
😜
💋
😄
😜
🙈
🙈
🙈
🙈
🔝
🌎
💪🏽
😍
😍
💜
💜
💜
💜
💕
💕
💕
☕
😍
☔
😍
😍

💯
💯
🔥
🌟
🙌
😄
🙂
😍
💛
🎼
🎤
😷
🤗
🙌
😎
💞
💞
💞
💞
🤗
🎉
⭐
⭐
🌎
😊
😋
😀
💖
🙈
😂
:)
🙌
😘
👋
🙈
🍆
🔥
🎥
😆
👇
👇
👇
💙
🙏
🐱
🐱
👨
👩🏼
💑
💏
👫
🙏
🔥
💪
🔥
🎬
🕯
🌹
☄
🐇
🐇
🗝
🗝
:)
😇
❗
🌊
💋
💋
💋
💋
💋
💋
💋
😊
💪🏼
😊
💞
😀
💰
⭐
⭐
🚨
😊
🎂
🎈
🙌
🌶
🌶
🌶
🔥
🔥
🔥
👽
🔥
👉
😀
⭐
⭐
😊
🌴
😁
😭
😭
😭
😭
:)
😂
🌟
📷
😭
😭
😭
😍
💜
🐼
💜
💜
😋
🙏🏾
🙏🏾
🙏🏾
🙏🏾
🙏🏾
🙏🏾
😜
🖐
🖐
😊
😇
😇
💕
🎊
🎂
🎊
💃🏽
🥂
🎈
🎂
😭
💧
💧
💥
🔥
💥
🍺
🍻
🌴
💋
💋
💋
💋
💀
🎢
🔞
😂
😶
🔊
💾
👇
👇
👇
🙏
🌟
🌟
🌟
🌟
🌟
🏋
📲
🙏🏻
😀
❌
❌
❌
😎
😠
😠
😠
😘
😘
🤣
🤣
🤣
🤣
🤣
🤣
🤣
🤣
🙏🏼
🙌
🎶
👍
😍
😍
😍
🌹
🌹
🌹
☕
🎶
🐦
🍃
🍁
🌿
🍂
🍃
🍁
🌿
🍂
🍃
🌹
🌹
🎂
👑
🎁
🎉
😍
💕
😊
😘
😏
😈
😜
😝
🤤
😋
😋
😭
😭
😍
😍
😍
🌷
⏳
😎
🔝
🎭
🚺
😈
🔥
🔥
🔥
🔥
😂
🙈
💘
⛸
😈
😈
🔥
🔥
😍
✋
🔃
🔃
🔃
🔃
🌙
😇
🙋
👏
🌹
🌹
🌹
☕
🎶
🐦
🍃
🍁
🌿
🍂
🍃
🍁
🌿
🍂
🍃
🌹
🌹
😀
😀
😑
🥃
🥃
🤔
🤔
🤔
✨
😍
✨
🔪
🔪
🔪
😳
💫
⚡
🌟
🌹
✨
🌹
✨
😋
👑
👑
🍑
😘
😂
😂
:)
👠
😁
📸
📷
😂
🔥
🎯
🎯
🔥
🔥
😭
🌎
🛑
😁
😁
😂
👍
😍
💕
🌷
🌳
🐑
👨
🌾
👨
🌾
🥕
🥔
🥕
💪🏻
✨
💄
😁
🐰
👉
📻
🎷
🎶
🔕
🎸
🎼
🤛
🐱
💤
👊
🎓
🍓
🙏🏽
🕊
💯
🔥
😘
😘
😘
😘
😘
🤗
🤗
🤗
🤗
🤗
👌🏼
👌🏼
💗
💗
🌻
🖨
🆓
👇
🌹
🌹
🌹
☕
🎶
🐦
🍃
🍁
🌿
🍂
🍃
🍁
🌿
🍂
🍃
🌹
🌹
👉
👈
😳
💦
🍑
🍰
🥘
🍷
🙁
🍵
🎉
🎉
🍵
🎭
🌹
👑
👑
💕
💙
🌷
🌷
💙
🌷
🌷
🌷
🌷
💙
💙
💋
💋
💋
💋
💋
💋
💋
💋
💟
🐘
🐲
🐪
🐝
😂
😂
😂
😂
😂
🌈
🎶
🎵
🎶
🎵
🎸
😂
😂
🎥
😩
👇
👇
👇
🔥
🔥
🔥
💚
👑
💚
🔥
🔥
🔥
🔥
🔥
🔥
🔥
🔥
🔥
🔥
🔥
💚
👑
💚
🔥
🔥
🔥
🔥
🔥
🔥
🔥
🔥
🔥
🔥
🔥
🔥
🔥
👇
👇
👇
👇

🙌
🙏🏽
😁
😫
😋
🌹
💖
💛
💖
🌹
💖
💛
💖
🌹
💖
💛
🌹
🌹
🌹
🌹
☕
🎶
🐦
🍃
🍁
🌿
🍂
🍃
🍁
🌿
🍂
🍃
🌹
🌹
📝
😄
👌
👌
💕
💞
⚽
🏆
🎊
◾
◾
◾
😍
🌹
🌹
🌹
☕
🎶
🐦
🍃
🍁
🌿
🍂
🍃
🍁
🌿
🍂
🍃
🌹
🌹
📰
👨
👶
😍
💕
✊
✊
✊🏾
👀
🆕
🎥
🌐
💕
😊
🇬🇧
🔝
👑
💝
😍
🔥
😍
🔥
😍
🚺
🌐
😊
⚽
:)
🙏🏽
🤗
😂
😆
😍
😘
😘
😘
💦
⏩
🔥
😍
🇺🇸
😍
⏩
⏩
⏩
⏪
⏪
⏪
👍
👍
😉
💋
👍
🏆
😘
😃
😁
😂
🙌
🙏🏽
🍃
🌹
💋
🌼
🌼
💋
💦
💦
🍃
🌹
🌹
🍃
🔥
🍍
🌴
🏖
😋
😏
🤤
👀
🍑
😍
🌻
💎
🌻
😂
❌
🔊
🔞
😳
🆘
😂
😂
😂
😂
😍
😍
💗
😝
🙌
💪🏼
🚗
✅
💸
🍀
🍀
😜
🌴
🌺
🇪🇸
😘
🍓
☕
🍵
🥃
🍷
💦
🔥
💯
😍
😃
😃
🔞
😯
👌
😂
🤗
😜
⏩
⏪
😁
🤗
🌻
🌞
✨
😂
😂
😂
😂
😂
🔥
🔥
🔥
🔥
🥗
🐖
🥔
😂
😂
😂
😂
😂
😎
😍
⚽
😂
🤣
😂
😂
💘
😍
😍
😍
💚
💚
💚
💚
😕
💕
💃🏽
😋
😋
👑
👑
💪
😘
🙅🏾
🙊
💦
😍
😱
👁
👁
💥
👏
👏
👏
🔝
👉
👈
💖
💖
💖
😔
🎨
🎞
💪🏼
😂
💚
🤣
😂
🤣
😏
😘
😍
😘
💕
📸
😍
🌈
🙏🏼
🌌
💜
💜
😉
✨
🕺
✨
💕
😈
🌏
🌏
🎬
🎥
💀
🎭
🍓
🌈
🍀
🌈
🍀
🌈
🍀
;)
🔥
😈
💃🏻
🎼
🦂
💋
💯
😁
💞
🐰
🐣
💀
👉
🤘
😉
💞
🙏
🌹
😈
💪🏻
👊🏻
🐷
🐽
🏀
🔥
😈
😈
😎
🔥
🔥
😍
🔥
👇
👇
👇
🌟
🌟
💋
💋
💋
💋
💋
💋
💋
😊
👾
😊
💗
💪
👪
😍
🙏
😛
👼🏻
⭐
😱
👌
💕
😍
😍
👏
🌷
😍
😉
😉
😏
😝
💛
:)
💋
💞
📲
😍
💫
😘
😘
😘
😘
😘
😘
😘
😘
😘
😘
😘
😘
😘
😘
😘
😘
😘
😘
😘
😘
👶
😀
➿
🔊
💖
😘
💖
💗
💝
😍
😙
😚
👑
💎
🌟
✊
🍆
💦
💦
💦
🔥
🔥
🔥
😜
😅
😂
🤣
👌
😘
😍
😂
😂
😂
🦋
💦
🌹
🔨
🎅
😊
😎
🙏
🌸
😍
🌺
😁
💪
👍🏾
😡
💫
😘
😘
😘
😘
😘
😘
😘
😘
😘
😘
😘
😘
😘
😘
😘
😘
😘
😘
😘
😘
💋
💋
💋
💋
💋
💋
💋
🌹
🌹
😍
😍
🌹
🌹
🍊
🍉
🍏
🥕
🍈
🥑
🍎
🍽
☕
🍵
😀
😍
😊
😁
😈
😈


# 1.2 Build ML models with all embeddings

In [17]:
X_train.shape,X_test.shape

((45957, 300), (12095, 300))

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,f1_score
from sklearn.linear_model import SGDClassifier

In [19]:
rf = RandomForestClassifier()
sgd = SGDClassifier()

In [20]:
rf.fit(X_train,y_train)
pred=rf.predict(X_test)
print('random forest:')
print("acc is {0:.2f}, f1 is {1:.2f}".format(accuracy_score(y_test,pred),
                                            f1_score(y_test,pred,average='weighted')))

random forest:
acc is 0.66, f1 is 0.64


In [23]:
pd.Series(pred).value_counts()

0    8581
1    3199
5     137
4     110
2      64
3       4
dtype: int64

In [28]:
for i in range(6):
    print(accuracy_score(y_test==i,pred==i),f1_score(y_test==i,pred==i))

0.6887143447705664 0.7621454292753806
0.7160810252170319 0.5040439052570769
0.9755270773046714 0.16853932584269662
0.9955353451839604 0.09999999999999999
0.9743695742042167 0.3231441048034934
0.9713104588673005 0.3836589698046181


In [23]:
import xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eval_metric'] = 'mlogloss'
param['num_class'] = 6
param
dtrain = xgboost.DMatrix(np.array(X_train), label=np.array(y_train))
dtest = xgboost.DMatrix(np.array(X_test), label=np.array(y_test))
deval = xgboost.DMatrix(np.array(X_dev), label=np.array(y_dev))

watchlist = [(dtrain, 'train'), (deval, 'eval')]
num_round = 5
bst = xgboost.train(param, dtrain, num_round, watchlist)


pred=bst.predict(dtest)
print("xgboost:")
print("acc is {0:.2f}, f1 is {1:.2f}".format(accuracy_score(y_test,pred),
                                            f1_score(y_test,pred,average='weighted')))

/Users/nihaozheng/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[0]	train-mlogloss:1.44575	eval-mlogloss:1.45367
[1]	train-mlogloss:1.25533	eval-mlogloss:1.27191
[2]	train-mlogloss:1.12814	eval-mlogloss:1.15278
[3]	train-mlogloss:1.03591	eval-mlogloss:1.06738
[4]	train-mlogloss:0.967395	eval-mlogloss:1.00799
xgboost:
acc is 0.64, f1 is 0.58


In [24]:
sgd.fit(X_train,y_train)
pred = sgd.predict(X_test)
print("SGD classifier:")
print("acc is {0:.2f}, f1 is {1:.2f}".format(accuracy_score(y_test,pred), f1_score(y_test,pred,average='weighted')))




/Users/nihaozheng/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGD classifier:
acc is 0.63, f1 is 0.60


# 1.3 parameter tuning

In [94]:
from sklearn.model_selection import RandomizedSearchCV


In [83]:
#xgboost
import xgboost
dtrain = xgboost.DMatrix(np.array(X_train), label=np.array(y_train))
dtest = xgboost.DMatrix(np.array(X_test), label=np.array(y_test))
deval = xgboost.DMatrix(np.array(X_dev), label=np.array(y_dev))
watchlist = [(dtrain,'train'),(deval,'val')]

In [100]:
cv_params = {'max_depth': [3,5,7,10,13,15,20,30,50,100], 'min_child_weight': [1,3,5,7,9,11,13,15],
     'learning_rate': [0.005,0.01,0.02,0.05,0.1,0.3,0.5],
     'n_estimators': [50,100,150,200,300,400,500],
     'subsample': [0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.7,0.8],
     'colsample_bytree': [0.1,0.15,0.2,0.5,0.6,0.7,0.8]
    }
ind_params = { 'seed':0}

optimized_GBM =RandomizedSearchCV(XGBClassifier(**ind_params), 
                        cv_params, 
                         scoring = 'neg_log_loss', cv = 3, n_jobs = -1) 
optimized_GBM.fit(np.array(X_train),np.array(y_train))
model = optimized_GBM.best_estimator_
score = optimized_GBM.best_score_


In [101]:
score

-1.0158784105872443

In [103]:
pred=model.predict(X_test)
print("xgboost:")
print("acc is {0:.2f}, f1 is {1:.2f}".format(accuracy_score(y_test,pred),
                                            f1_score(y_test,pred,average='weighted')))

xgboost:
acc is 0.60, f1 is 0.45


/Users/nihaozheng/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# 2 no emoji embedding

In [25]:
import time
X_train,y_train = train['text'],train['label']
X_test,y_test = test['text'],test['label']
X_dev,y_dev =dev['text'],dev['label']

In [26]:
# set all unseen word to 0
# set all emoji to 0
def individual2vec(word):
    #print(word)
    if word in emoticon2vec.vocab:
        print(word)
        return emoticon2vec[word]
    elif word in reverse_emoji_dict.keys():
        return np.zeros(300)
            
    elif word in word2vec.vocab:
        return word2vec[word]
    else:
        return np.zeros(300)

    

In [27]:
def sentence2vec(s):
    #sum over the embedding
    s = s.split()
    return sum(list(map(individual2vec,s)))

In [28]:
def data2vec(data):
    return list(map(sentence2vec,data))

In [29]:
start = time.time()

X_train = data2vec(X_train)
X_test = data2vec(X_test)
X_dev=data2vec(X_dev)

end = time.time()
print("the transformation takes {0:.2f} minutes".format((end-start)/60 ))

:)
:)
:)
;)
:)
8)
8)
:)
:)
:)
:)
:)
:)
:)
;)
:)
;)
:)
:)
:)
;)
:)
:)
:)
:)
:)
=)
:)
:)
:)
8)
:)
:)
:)
;)
8-)
:)
:)
:)
;)
:)
:)
;)
:)
:)
:)
:(
:)
:(
:)
:)
:)
:)
:)
:)
;)
:)
:)
;)
;)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
;)
:)
:)
:)
:)
;)
:)
:)
:)
:)
:)
:)
:(
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
d:
:)
:)
:)
:)
:)
:)
:)
;)
:)
:)
;)
:)
:)
:)
:)
;)
:)
;)
;)
:)
;)
:)
:)
:)
:)
:)
:)
:')
;)
:)
:o
:)
;)
:)
:)
:)
:(
:)
:)
:)
:)
:)
:)
:)
*)
;)
:)
:(
;)
:)
:)
:)
:)
:)
;)
:)
;)
:)
:)
:)
:)
;)
:)
:)
:)
:)
:)
:)
:)
;)
;)
:)
:)
8-)
:(
;)
:)
:)
:)
:)
:)
:)
:p
;)
:)
:)
:)
;)
:)
:)
;)
:)
:)
:)
:)
:)
:)
:)
:)
;)
;)
:)
;)
:)
:)
:)
:)
:)
:)
:)
:)
:)
;)
:)
;)
:)
:)
;)
:)
:)
:)
:)
:)
:)
;)
:)
;)
;)
:)
;)
;)
:(
:)
:)
;)
:)
:)
:)
;)
:)
:)
:)
:)
:)
:)
:)
:)
:(
;)
:)
:p
:)
:)
;)
:)
:)
:)
:)
:)
:)
:)
:)
;)
:)
;)
:)
:)
:)
:)
:)
:)
:p
;)
:)
:)
:)
:)
:)
;)
;)
:)
:)
:)
;)
;)
:)
:)
:p
:p
;)
:)
;)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
;)
:)
;)
:)
:)
:)
:)
:)
;)
:)
:)
:)
:)
;

# 2.1 ML without emoji embedding

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score,f1_score

In [31]:
rf = RandomForestClassifier()
sgd = SGDClassifier()

In [32]:
start = time.time()
rf.fit(X_train,y_train)
pred=rf.predict(X_test)
print('random forest:')
print("acc is {0:.2f}, f1 is {1:.2f}".format(accuracy_score(y_test,pred),
                                            f1_score(y_test,pred,average='weighted')))
end = time.time()
print("rf takes {0:.2f} minutes".format((end-start)/60 ))

random forest:
acc is 0.64, f1 is 0.61
rf takes 0.18 minutes


In [33]:
import xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eval_metric'] = 'mlogloss'
param['num_class'] = 6
param
dtrain = xgboost.DMatrix(np.array(X_train), label=np.array(y_train))
dtest = xgboost.DMatrix(np.array(X_test), label=np.array(y_test))
deval = xgboost.DMatrix(np.array(X_dev), label=np.array(y_dev))

watchlist = [(dtrain, 'train'), (deval, 'eval')]
num_round = 5
bst = xgboost.train(param, dtrain, num_round, watchlist)


pred=bst.predict(dtest)
print("xgboost:")
print("acc is {0:.2f}, f1 is {1:.2f}".format(accuracy_score(y_test,pred),
                                            f1_score(y_test,pred,average='weighted')))

[0]	train-mlogloss:1.45518	eval-mlogloss:1.46368
[1]	train-mlogloss:1.27087	eval-mlogloss:1.28886
[2]	train-mlogloss:1.14754	eval-mlogloss:1.17234
[3]	train-mlogloss:1.05797	eval-mlogloss:1.09143
[4]	train-mlogloss:0.991649	eval-mlogloss:1.03413
xgboost:
acc is 0.63, f1 is 0.56


/Users/nihaozheng/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [34]:
sgd.fit(X_train,y_train)
pred = sgd.predict(X_test)
print("SGD classifier:")
print("acc is {0:.2f}, f1 is {1:.2f}".format(accuracy_score(y_test,pred), f1_score(y_test,pred,average='weighted')))





/Users/nihaozheng/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGD classifier:
acc is 0.58, f1 is 0.57


# 2.3 parameter tuning

In [24]:
from sklearn.model_selection import RandomizedSearchCV

start = time.time()
best = RandomForestClassifier(random_state=40)

forest_reg = RandomForestClassifier(random_state=40,n_jobs=-1)
param_dist = {"max_depth": [10,20,30,40,50,60,70,80,90,100,200,300],
              'n_estimators':[50,100,150,200,300,400,500,600,700,1000],
            "max_features": [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
            "min_samples_split": [2,4,6,8,10,15,20],
                "min_samples_leaf": [2,4,6,8,10,15,20]

             }
rand_search = RandomizedSearchCV(forest_reg,param_dist,cv=3,scoring='neg_log_loss',verbose=False)
rand_search.fit(X_train,y_train)
model = rand_search.best_estimator_

                                        
end = time.time()
print("cv takes {0:.2f} minutes".format((end-start)/60 ))

cv takes 283.86 minutes


In [25]:
pred = model.predict(X_test)
print("rf with cv:")
print("acc is {0:.2f}, f1 is {1:.2f}".format(accuracy_score(y_test,pred),
                                              f1_score(y_test,pred,average='weighted')))




rf with cv:
acc is 0.65, f1 is 0.55


/Users/nihaozheng/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# 3.1 no emoticon embedding

In [35]:
X_train,y_train = train['text'],train['label']
X_test,y_test = test['text'],test['label']
X_dev,y_dev =dev['text'],dev['label']

In [36]:
# set all unseen word to 0
def individual2vec(word):
    #print(word)
    if word in emoticon2vec.vocab:
        print(word)
        return np.zeros(300)
    elif word in reverse_emoji_dict.keys():
        d = reverse_emoji_dict[word]
        d = d.encode('utf-8')
        d = d.decode('unicode-escape')
        if d in emoji2vec.vocab:
            #print(d)
            return emoji2vec[d]
        else:
            return np.zeros(300)
            
    elif word in word2vec.vocab:
        return word2vec[word]
    else:
        return np.zeros(300)

    

In [37]:
def sentence2vec(s):
    #sum over the embedding
    s = s.split()
    return sum(list(map(individual2vec,s)))

In [38]:
def data2vec(data):
    return np.array(list((map(sentence2vec,data))))

In [39]:
start = time.time()

X_train = data2vec(X_train)
X_test = data2vec(X_test)
X_dev=data2vec(X_dev)

end = time.time()
print("the transformation takes {0:.2f} minutes".format((end-start)/60 ))

:)
:)
:)
;)
:)
8)
8)
:)
:)
:)
:)
:)
:)
:)
;)
:)
;)
:)
:)
:)
;)
:)
:)
:)
:)
:)
=)
:)
:)
:)
8)
:)
:)
:)
;)
8-)
:)
:)
:)
;)
:)
:)
;)
:)
:)
:)
:(
:)
:(
:)
:)
:)
:)
:)
:)
;)
:)
:)
;)
;)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
;)
:)
:)
:)
:)
;)
:)
:)
:)
:)
:)
:)
:(
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
d:
:)
:)
:)
:)
:)
:)
:)
;)
:)
:)
;)
:)
:)
:)
:)
;)
:)
;)
;)
:)
;)
:)
:)
:)
:)
:)
:)
:')
;)
:)
:o
:)
;)
:)
:)
:)
:(
:)
:)
:)
:)
:)
:)
:)
*)
;)
:)
:(
;)
:)
:)
:)
:)
:)
;)
:)
;)
:)
:)
:)
:)
;)
:)
:)
:)
:)
:)
:)
:)
;)
;)
:)
:)
8-)
:(
;)
:)
:)
:)
:)
:)
:)
:p
;)
:)
:)
:)
;)
:)
:)
;)
:)
:)
:)
:)
:)
:)
:)
:)
;)
;)
:)
;)
:)
:)
:)
:)
:)
:)
:)
:)
:)
;)
:)
;)
:)
:)
;)
:)
:)
:)
:)
:)
:)
;)
:)
;)
;)
:)
;)
;)
:(
:)
:)
;)
:)
:)
:)
;)
:)
:)
:)
:)
:)
:)
:)
:)
:(
;)
:)
:p
:)
:)
;)
:)
:)
:)
:)
:)
:)
:)
:)
;)
:)
;)
:)
:)
:)
:)
:)
:)
:p
;)
:)
:)
:)
:)
:)
;)
;)
:)
:)
:)
;)
;)
:)
:)
:p
:p
;)
:)
;)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
;)
:)
;)
:)
:)
:)
:)
:)
;)
:)
:)
:)
:)
;

# 3.2 ML without emoticon

In [40]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score,f1_score

In [41]:
rf = RandomForestClassifier()
sgd = SGDClassifier()

In [42]:
start = time.time()
rf.fit(X_train,y_train)
pred=rf.predict(X_test)
print('random forest:')
print("acc is {0:.2f}, f1 is {1:.2f}".format(accuracy_score(y_test,pred),
                                            f1_score(y_test,pred,average='weighted')))
end = time.time()
print("rf takes {0:.2f} minutes".format((end-start)/60 ))

random forest:
acc is 0.66, f1 is 0.64
rf takes 0.18 minutes


In [243]:
import xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eval_metric'] = 'mlogloss'
param['num_class'] = 6
param
dtrain = xgboost.DMatrix(np.array(X_train), label=np.array(y_train))
dtest = xgboost.DMatrix(np.array(X_test), label=np.array(y_test))
deval = xgboost.DMatrix(np.array(X_dev), label=np.array(y_dev))

watchlist = [(dtrain, 'train'), (deval, 'eval')]
num_round = 5
bst = xgboost.train(param, dtrain, num_round, watchlist)


pred=bst.predict(dtest)
print("xgboost:")
print("acc is {0:.2f}, f1 is {1:.2f}".format(accuracy_score(y_test,pred),
                                            f1_score(y_test,pred,average='weighted')))

[0]	train-mlogloss:1.43509	eval-mlogloss:1.45808
[1]	train-mlogloss:1.23399	eval-mlogloss:1.27208
[2]	train-mlogloss:1.09564	eval-mlogloss:1.14803
[3]	train-mlogloss:0.994125	eval-mlogloss:1.05872
[4]	train-mlogloss:0.914151	eval-mlogloss:0.995001
xgboost:
acc is 0.66, f1 is 0.60


In [248]:
sgd.fit(X_train,y_train)
pred = sgd.predict(X_test)
print("SGD classifier:")
print("acc is {0:.2f}, f1 is {1:.2f}".format(accuracy_score(y_test,pred), f1_score(y_test,pred,average='weighted')))






/Users/nihaozheng/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGD classifier:
acc is 0.61, f1 is 0.53


/Users/nihaozheng/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# 4.1 No emoji/emoticon

In [43]:
X_train,y_train = train['text'],train['label']
X_test,y_test = test['text'],test['label']
X_dev,y_dev =dev['text'],dev['label']

In [44]:
# set all unseen word to 0
def individual2vec(word):
    #print(word)
    if word in emoticon2vec.vocab:
        return np.zeros(300)
    elif word in reverse_emoji_dict.keys():
        return np.zeros(300)
            
    elif word in word2vec.vocab:
        return word2vec[word]
    else:
        return np.zeros(300)

    

In [45]:
def sentence2vec(s):
    #sum over the embedding
    s = s.split()
    return sum(list(map(individual2vec,s)))

In [46]:
def data2vec(data):
    return np.array(list((map(sentence2vec,data))))

In [47]:
start = time.time()

X_train = data2vec(X_train)
X_test = data2vec(X_test)
X_dev=data2vec(X_dev)

end = time.time()
print("the transformation takes {0:.2f} minutes".format((end-start)/60 ))

the transformation takes 0.13 minutes


# 4.2 ML without emoji/emoticon

In [48]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score,f1_score

In [49]:
rf = RandomForestClassifier()
sgd = SGDClassifier()

In [50]:
start = time.time()
rf.fit(X_train,y_train)
pred=rf.predict(X_test)
print('random forest:')
print("acc is {0:.2f}, f1 is {1:.2f}".format(accuracy_score(y_test,pred),
                                            f1_score(y_test,pred,average='weighted')))
end = time.time()
print("rf takes {0:.2f} minutes".format((end-start)/60 ))

random forest:
acc is 0.65, f1 is 0.62
rf takes 0.16 minutes


In [51]:
import xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eval_metric'] = 'mlogloss'
param['num_class'] = 6
param
dtrain = xgboost.DMatrix(np.array(X_train), label=np.array(y_train))
dtest = xgboost.DMatrix(np.array(X_test), label=np.array(y_test))
deval = xgboost.DMatrix(np.array(X_dev), label=np.array(y_dev))

watchlist = [(dtrain, 'train'), (deval, 'eval')]
num_round = 5
bst = xgboost.train(param, dtrain, num_round, watchlist)


pred=bst.predict(dtest)
print("xgboost:")
print("acc is {0:.2f}, f1 is {1:.2f}".format(accuracy_score(y_test,pred),
                                            f1_score(y_test,pred,average='weighted')))

[0]	train-mlogloss:1.45445	eval-mlogloss:1.46211
[1]	train-mlogloss:1.27027	eval-mlogloss:1.2846
[2]	train-mlogloss:1.14749	eval-mlogloss:1.17092
[3]	train-mlogloss:1.05836	eval-mlogloss:1.09007
[4]	train-mlogloss:0.991719	eval-mlogloss:1.03203
xgboost:
acc is 0.63, f1 is 0.56


/Users/nihaozheng/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [52]:
sgd.fit(X_train,y_train)
pred = sgd.predict(X_test)
print("SGD classifier:")
print("acc is {0:.2f}, f1 is {1:.2f}".format(accuracy_score(y_test,pred), f1_score(y_test,pred,average='weighted')))







/Users/nihaozheng/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGD classifier:
acc is 0.60, f1 is 0.57


/Users/nihaozheng/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Use new embedding

In [53]:
from gensim.models.keyedvectors import KeyedVectors
all_embed= KeyedVectors.load_word2vec_format('./embedding/all_embedding.txt', binary=False)



In [54]:
#load dictionary
import pickle
emoji_dict =pickle.load(open('./data/SST2/emoji_dict.p','rb'))
print("emoji dictionary load successfully")

emoji dictionary load successfully


In [55]:
import pandas as pd
import numpy as np
train = pd.read_csv('./data/SST2/train.csv')
dev = pd.read_csv('./data/SST2/dev.csv')
test = pd.read_csv('./data/SST2/test.csv')

In [56]:
#build reverse dict, i.e emoji*** : \U****
reverse_emoji_dict = {}
for key,item in emoji_dict.items():
    reverse_emoji_dict[item]=key

# 1 Transform input to embedding

In [57]:
import time

In [58]:
X_train,y_train = train['text'],train['label']
X_test,y_test = test['text'],test['label']
X_dev,y_dev =dev['text'],dev['label']

In [62]:
emoji_dict['\\U0001f30c']

'emoji0'

In [63]:
X_train[0]

"While I'm breathing - I love and believe. emoji1334 "

In [60]:
all_emoji = emoji_dict.keys()
all_emoticon = [' :) ', ' =) ', ' :( ', ' ;P ', ' ;) ', ' :-D ', ' :D ', ' :-) ', ' ;( ', ' :/ ', ' (: ', ' :-/ ', ' :P ', ':|', ' xD ', " :') ", ' =D ',  ' :] ', ' ;-) ', ' :p ', ' DX ', " :'( ", ' :-P ', ' :@ ', ' =] ', ' :-( ', ' >:( ', ' ;D ', ' XP ', ' 8) ', ' :\\ ',' XD ']
all_emoticon = [i.strip() for i in all_emoticon]


In [64]:
# set all unseen word to 0
def individual2vec(word):
    #print(word)
    if word in all_embed.vocab:
        return all_embed[word]
    else:
        return np.zeros(300)

def sentence2vec(s):
    #sum over the embedding
    s = s.split()
    return sum(list(map(individual2vec,s)))
def data2vec(data):
    return np.array(list((map(sentence2vec,data))))

In [65]:
start = time.time()

X_train = data2vec(X_train)
X_test = data2vec(X_test)
X_dev=data2vec(X_dev)

end = time.time()
print("the transformation takes {0:.2f} minutes".format((end-start)/60 ))

the transformation takes 0.13 minutes


In [66]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,f1_score
from sklearn.linear_model import SGDClassifier

In [67]:
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
pred=rf.predict(X_test)
print('random forest:')
print("acc is {0:.2f}, f1 is {1:.2f}".format(accuracy_score(y_test,pred),
                                            f1_score(y_test,pred,average='weighted')))

random forest:
acc is 0.65, f1 is 0.63


In [69]:
sgd = SGDClassifier()
sgd.fit(X_train,y_train)
pred = sgd.predict(X_test)
print('sgd:')
print("acc is {0:.2f}, f1 is {1:.2f}".format(accuracy_score(y_test,pred),
                                            f1_score(y_test,pred,average='weighted')))

/Users/nihaozheng/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


sgd:
acc is 0.57, f1 is 0.57


In [70]:
import xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eval_metric'] = 'mlogloss'
param['num_class'] = 6
param
dtrain = xgboost.DMatrix(np.array(X_train), label=np.array(y_train))
dtest = xgboost.DMatrix(np.array(X_test), label=np.array(y_test))
deval = xgboost.DMatrix(np.array(X_dev), label=np.array(y_dev))

watchlist = [(dtrain, 'train'), (deval, 'eval')]
num_round = 5
bst = xgboost.train(param, dtrain, num_round, watchlist)


pred=bst.predict(dtest)
print("xgboost:")
print("acc is {0:.2f}, f1 is {1:.2f}".format(accuracy_score(y_test,pred),
                                            f1_score(y_test,pred,average='weighted')))

[0]	train-mlogloss:1.44141	eval-mlogloss:1.4464
[1]	train-mlogloss:1.25493	eval-mlogloss:1.26577
[2]	train-mlogloss:1.13321	eval-mlogloss:1.14896
[3]	train-mlogloss:1.04735	eval-mlogloss:1.06877
[4]	train-mlogloss:0.984028	eval-mlogloss:1.01148
xgboost:
acc is 0.64, f1 is 0.59


# 2 No Emoji Embedding

In [73]:
# set all unseen word to 0
def individual2vec(word):
    #print(word)
    if word in all_embed.vocab:
        if word in reverse_emoji_dict.keys():
            return np.zeros(300)
        else:
            return all_embed[word]
    else:
        return np.zeros(300)

def sentence2vec(s):
    #sum over the embedding
    s = s.split()
    return sum(list(map(individual2vec,s)))
def data2vec(data):
    return np.array(list((map(sentence2vec,data))))

In [74]:
X_train,y_train = train['text'],train['label']
X_test,y_test = test['text'],test['label']
X_dev,y_dev =dev['text'],dev['label']
start = time.time()

X_train = data2vec(X_train)
X_test = data2vec(X_test)
X_dev=data2vec(X_dev)

end = time.time()
print("the transformation takes {0:.2f} minutes".format((end-start)/60 ))

the transformation takes 0.12 minutes


In [75]:
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
pred=rf.predict(X_test)
print('random forest:')
print("acc is {0:.2f}, f1 is {1:.2f}".format(accuracy_score(y_test,pred),
                                            f1_score(y_test,pred,average='weighted')))

random forest:
acc is 0.63, f1 is 0.60


In [76]:
sgd = SGDClassifier()
sgd.fit(X_train,y_train)
pred = sgd.predict(X_test)
print('sgd:')
print("acc is {0:.2f}, f1 is {1:.2f}".format(accuracy_score(y_test,pred),
                                            f1_score(y_test,pred,average='weighted')))

/Users/nihaozheng/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


sgd:
acc is 0.60, f1 is 0.50


/Users/nihaozheng/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [77]:
import xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eval_metric'] = 'mlogloss'
param['num_class'] = 6
param
dtrain = xgboost.DMatrix(np.array(X_train), label=np.array(y_train))
dtest = xgboost.DMatrix(np.array(X_test), label=np.array(y_test))
deval = xgboost.DMatrix(np.array(X_dev), label=np.array(y_dev))

watchlist = [(dtrain, 'train'), (deval, 'eval')]
num_round = 5
bst = xgboost.train(param, dtrain, num_round, watchlist)


pred=bst.predict(dtest)
print("xgboost:")
print("acc is {0:.2f}, f1 is {1:.2f}".format(accuracy_score(y_test,pred),
                                            f1_score(y_test,pred,average='weighted')))

[0]	train-mlogloss:1.46313	eval-mlogloss:1.46952
[1]	train-mlogloss:1.28649	eval-mlogloss:1.29828
[2]	train-mlogloss:1.16893	eval-mlogloss:1.18696
[3]	train-mlogloss:1.08662	eval-mlogloss:1.11074
[4]	train-mlogloss:1.02634	eval-mlogloss:1.05526
xgboost:
acc is 0.62, f1 is 0.54


/Users/nihaozheng/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# 3. No emoticon Embedding

In [78]:
# set all unseen word to 0
def individual2vec(word):
    #print(word)
    if word in all_embed.vocab:
        if word in all_emoticon:
            return np.zeros(300)
        else:
            return all_embed[word]
    else:
        return np.zeros(300)

def sentence2vec(s):
    #sum over the embedding
    s = s.split()
    return sum(list(map(individual2vec,s)))
def data2vec(data):
    return np.array(list((map(sentence2vec,data))))

In [79]:
X_train,y_train = train['text'],train['label']
X_test,y_test = test['text'],test['label']
X_dev,y_dev =dev['text'],dev['label']
start = time.time()

X_train = data2vec(X_train)
X_test = data2vec(X_test)
X_dev=data2vec(X_dev)

end = time.time()
print("the transformation takes {0:.2f} minutes".format((end-start)/60 ))

the transformation takes 0.12 minutes


In [80]:
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
pred=rf.predict(X_test)
print('random forest:')
print("acc is {0:.2f}, f1 is {1:.2f}".format(accuracy_score(y_test,pred),
                                            f1_score(y_test,pred,average='weighted')))

random forest:
acc is 0.65, f1 is 0.63


In [81]:
sgd = SGDClassifier()
sgd.fit(X_train,y_train)
pred = sgd.predict(X_test)
print('sgd:')
print("acc is {0:.2f}, f1 is {1:.2f}".format(accuracy_score(y_test,pred),
                                            f1_score(y_test,pred,average='weighted')))

/Users/nihaozheng/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


sgd:
acc is 0.60, f1 is 0.45


/Users/nihaozheng/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [82]:
import xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eval_metric'] = 'mlogloss'
param['num_class'] = 6
param
dtrain = xgboost.DMatrix(np.array(X_train), label=np.array(y_train))
dtest = xgboost.DMatrix(np.array(X_test), label=np.array(y_test))
deval = xgboost.DMatrix(np.array(X_dev), label=np.array(y_dev))

watchlist = [(dtrain, 'train'), (deval, 'eval')]
num_round = 5
bst = xgboost.train(param, dtrain, num_round, watchlist)


pred=bst.predict(dtest)
print("xgboost:")
print("acc is {0:.2f}, f1 is {1:.2f}".format(accuracy_score(y_test,pred),
                                            f1_score(y_test,pred,average='weighted')))

[0]	train-mlogloss:1.44144	eval-mlogloss:1.44644
[1]	train-mlogloss:1.25438	eval-mlogloss:1.26468
[2]	train-mlogloss:1.1328	eval-mlogloss:1.14971
[3]	train-mlogloss:1.04684	eval-mlogloss:1.06926
[4]	train-mlogloss:0.984004	eval-mlogloss:1.0131
xgboost:
acc is 0.64, f1 is 0.59


# 4. No embedding 

In [84]:
# set all unseen word to 0
def individual2vec(word):
    #print(word)
    if word in all_embed.vocab:
        if word in all_emoticon or word in reverse_emoji_dict.keys():
            return np.zeros(300)
        else:
            return all_embed[word]
    else:
        return np.zeros(300)

def sentence2vec(s):
    #sum over the embedding
    s = s.split()
    return sum(list(map(individual2vec,s)))
def data2vec(data):
    return np.array(list((map(sentence2vec,data))))

In [85]:
X_train,y_train = train['text'],train['label']
X_test,y_test = test['text'],test['label']
X_dev,y_dev =dev['text'],dev['label']
start = time.time()

X_train = data2vec(X_train)
X_test = data2vec(X_test)
X_dev=data2vec(X_dev)

end = time.time()
print("the transformation takes {0:.2f} minutes".format((end-start)/60 ))

the transformation takes 0.13 minutes


In [86]:
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
pred=rf.predict(X_test)
print('random forest:')
print("acc is {0:.2f}, f1 is {1:.2f}".format(accuracy_score(y_test,pred),
                                            f1_score(y_test,pred,average='weighted')))

random forest:
acc is 0.63, f1 is 0.60


In [87]:
sgd = SGDClassifier()
sgd.fit(X_train,y_train)
pred = sgd.predict(X_test)
print('sgd:')
print("acc is {0:.2f}, f1 is {1:.2f}".format(accuracy_score(y_test,pred),
                                            f1_score(y_test,pred,average='weighted')))

/Users/nihaozheng/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


sgd:
acc is 0.60, f1 is 0.45


/Users/nihaozheng/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [88]:
import xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eval_metric'] = 'mlogloss'
param['num_class'] = 6
param
dtrain = xgboost.DMatrix(np.array(X_train), label=np.array(y_train))
dtest = xgboost.DMatrix(np.array(X_test), label=np.array(y_test))
deval = xgboost.DMatrix(np.array(X_dev), label=np.array(y_dev))

watchlist = [(dtrain, 'train'), (deval, 'eval')]
num_round = 5
bst = xgboost.train(param, dtrain, num_round, watchlist)


pred=bst.predict(dtest)
print("xgboost:")
print("acc is {0:.2f}, f1 is {1:.2f}".format(accuracy_score(y_test,pred),
                                            f1_score(y_test,pred,average='weighted')))

[0]	train-mlogloss:1.46351	eval-mlogloss:1.47005
[1]	train-mlogloss:1.28561	eval-mlogloss:1.29709
[2]	train-mlogloss:1.1679	eval-mlogloss:1.18547
[3]	train-mlogloss:1.0854	eval-mlogloss:1.10788
[4]	train-mlogloss:1.02559	eval-mlogloss:1.05326
xgboost:
acc is 0.62, f1 is 0.54


/Users/nihaozheng/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Feed different numbers of data

In [19]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((45957, 300), (45957,), (12095, 300), (12095,))

In [20]:
for i in np.arange(0.1,1,0.1):
    print(i)

0.1
0.2
0.30000000000000004
0.4
0.5
0.6
0.7000000000000001
0.8
0.9


In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score,f1_score
from sklearn.model_selection import train_test_split

In [32]:
for i in np.arange(0.1,1.1,0.1):
    X_train_sub,temp1, y_train_sub, temp2 = train_test_split(X_train,y_train,test_size=1-i,stratify=y_train)
    print(X_train_sub.shape,y_train_sub.shape)
    rf = RandomForestClassifier()
    rf.fit(X_train_sub,y_train_sub)
    pred=rf.predict(X_test)
    print('random forest with {} X_train'.format(i))
    print("acc is {0:.2f}, f1 is {1:.2f}".format(accuracy_score(y_test,pred),
                                                f1_score(y_test,pred,average='weighted')))

(4595, 300) (4595,)


Exception: Data must be 1-dimensional

In [44]:
y_test.value_counts()

0    7248
1    3725
5     426
4     348
2     292
3      56
Name: label, dtype: int64